In [1]:
import collections
import sys
import numpy
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, SimpleRNN

Using TensorFlow backend.


In [2]:
from keras.layers import Merge
from keras.layers import Lambda
from keras import backend as K
import keras

In [3]:
questions = open('abstract_math_questions.txt','r')
labels = open('abstract_math_labels.txt','r')
info_sentences = open('abstract_math_info.txt','r')
fact_sentences = open('abstract_math_facts.txt','r')
standard_relations = open('abstract_math_relations.txt','r')

questions_list = []
labels_list = []
info_list = []
relations_list = []
facts_list = []

for sentence in questions:
    questions_list.append(sentence)
    
for sentence in labels:
    labels_list.append(sentence)
    
for sentence in info_sentences:
    info_list.append(sentence)

for sentence in fact_sentences:
    facts_list.append(sentence)
    
for sentence in standard_relations:
    relations_list.append(sentence)

In [4]:
print(len(questions_list[0]))
print(len(labels_list[0]))
print(len(info_list[0]))
print(len(facts_list[0]))
print(len(relations_list[0]))

24
10
26
5
30


In [5]:
#in this version, we put placeholders for numbers, such as QN1, QN2, etc.. for question numbers

fact_num_dict_list = []
ques_num_dict_list = []
info_num_dict_list = []
rela_num_dict_list = []

changed_facts_list = []
changed_questions_list = []
changed_info_list = []
changed_relations_list = []

#store the numbers in a list of dicts
#for facts list: (we will only process the info list, as others do not have numbers this time)
for sentence in facts_list:
    fact_num_dict = {}
    changed_fact_sentence = []
    indx_num = 1
    for word in sentence.split():
        changed_fact_sentence.append(word)
    changed_facts_list.append(changed_fact_sentence)
    fact_num_dict_list.append(fact_num_dict)

#repeat for questions
for sentence in questions_list:
    quest_num_dict = {}
    changed_qu_sentence = []
    indx_num = 1
    for word in sentence.split():
        changed_qu_sentence.append(word)
    changed_questions_list.append(changed_qu_sentence)
    ques_num_dict_list.append(quest_num_dict)   

#repeat for information
for sentence in info_list:
    info_num_dict = {}
    changed_info_sentence = []
    indx_num = 1
    for word in sentence.split():
        if word.isdigit():
            if indx_num == 1:
                new_word = "IN1"
            elif indx_num == 2:
                new_word = "IN2"
            elif indx_num == 3:
                new_word = "IN3"
            elif indx_num == 4:
                new_word = "IN4"
            elif indx_num == 5:
                new_word = "IN5"
            info_num_dict[word] = new_word
            changed_info_sentence.append(new_word)
            indx_num += 1
        else:
            changed_info_sentence.append(word)
    changed_info_list.append(changed_info_sentence)
    info_num_dict_list.append(info_num_dict)   

#repeat for information
for sentence in relations_list:
    rela_num_dict = {}
    changed_rela_sentence = []
    indx_num = 1
    for word in sentence.split():
        changed_rela_sentence.append(word)
    changed_relations_list.append(changed_rela_sentence)
    rela_num_dict_list.append(info_num_dict)   
    
facts_list = changed_facts_list
questions_list = changed_questions_list
info_list = changed_info_list
relations_list = changed_relations_list

In [6]:
list_of_all_variables = []

### Let's make a list of all variables and prepare them for one-hot-coding

# Start by assigning numbers to the standard variables:

list_of_all_variables.append("QN1")
list_of_all_variables.append("QN2")
list_of_all_variables.append("QN3")
list_of_all_variables.append("QN4")
list_of_all_variables.append("QN5")
list_of_all_variables.append("QN6")
list_of_all_variables.append("QN7")
list_of_all_variables.append("QN8")
list_of_all_variables.append("FN1")
list_of_all_variables.append("FN2")
list_of_all_variables.append("FN3")
list_of_all_variables.append("FN4")
list_of_all_variables.append("FN5")
list_of_all_variables.append("FN6")
list_of_all_variables.append("FN7")
list_of_all_variables.append("FN8")
list_of_all_variables.append("IN1")
list_of_all_variables.append("IN2")
list_of_all_variables.append("IN3")
list_of_all_variables.append("IN4")
list_of_all_variables.append("IN5")
list_of_all_variables.append("IN6")
list_of_all_variables.append("IN7")
list_of_all_variables.append("IN8")
list_of_all_variables.append("N1")
list_of_all_variables.append("N2")
list_of_all_variables.append("N3")
list_of_all_variables.append("N4")
list_of_all_variables.append("N5")
list_of_all_variables.append("N6")
list_of_all_variables.append("N7")
list_of_all_variables.append("N8")
list_of_all_variables.append("N9")
list_of_all_variables.append("V1")
list_of_all_variables.append("V2")
list_of_all_variables.append("V3")
list_of_all_variables.append("V4")
list_of_all_variables.append("V5")
list_of_all_variables.append("V6")
list_of_all_variables.append("V7")
list_of_all_variables.append("V8")
list_of_all_variables.append("V9")

for sentence in facts_list:
    for word in sentence:
        if word not in list_of_all_variables:
            list_of_all_variables.append(word)
    
for sentence in info_list:
    for word in sentence:
        if word not in list_of_all_variables:
            list_of_all_variables.append(word) 

for sentence in questions_list:
    for word in sentence:
        if word not in list_of_all_variables:
            list_of_all_variables.append(word)

for sentence in relations_list:
    for word in sentence:
        if word not in list_of_all_variables:
            list_of_all_variables.append(word)      
            
list_of_all_variables.append("<PAD>")

In [7]:
word_to_index = {}
index_to_word = {}

word_index = 0

for word in list_of_all_variables:
    word_to_index[word] = word_index
    index_to_word[word_index] = word
    word_index += 1
print(list_of_all_variables)

['QN1', 'QN2', 'QN3', 'QN4', 'QN5', 'QN6', 'QN7', 'QN8', 'FN1', 'FN2', 'FN3', 'FN4', 'FN5', 'FN6', 'FN7', 'FN8', 'IN1', 'IN2', 'IN3', 'IN4', 'IN5', 'IN6', 'IN7', 'IN8', 'N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8', 'N9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'NONE', 'MEANS', 'MORE', 'return', 'MAKES', 'ADD', ';', 'buy', 'SUBTRACT', 'changed', 'PAST-TENSE', 'change', 'ADD-OR-SUBTRACT', 'PLURAL', 'A1', 'opened', 'open', 'IN', 'HAVE', 'there', 'are', 'and', '.', 'in', 'the', ',', 'UNITS', 'of', 'at', 'with', 'next', 'to', 'on', 'other', 'side', 'from', 'has', 'already', 'one', 'were', 'a', 'comes', 'he', 'bought', 'last', 'week', 'yesterday', 'called', 'add', 'more', 'subtract', 'went', 'could', 'not', 'go', 'for', 'as', 'another', 'each', 'some', 'first', 'they', 'second', 'during', 'had', 'month', 'than', 'men', 'women', 'days', 'it', 'will', 'be', 'capacity', 'full', 'how', 'many', '?', 'when', 'did', 'total', 'was', 'an', 'have', 'been', 'two', 'KIND-OF', 'NOT', 'BEF

In [8]:
new_facts_list = []
new_questions_list = []
new_info_list = []
new_relations_list = []

pad_up_to = 100
pad_index = word_to_index["<PAD>"]

for sentence in facts_list:
    substitute_sentence = []
    for word in sentence:
        substitute_sentence.append(word_to_index[word])
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
    new_facts_list.append(substitute_sentence)

for sentence in questions_list:
    substitute_sentence = []
    for word in sentence:
        substitute_sentence.append(word_to_index[word])
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
    new_questions_list.append(substitute_sentence)
        
for sentence in info_list:
    substitute_sentence = []
    for word in sentence:
        substitute_sentence.append(word_to_index[word])
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
    new_info_list.append(substitute_sentence)

for sentence in relations_list:
    substitute_sentence = []
    for word in sentence:
        substitute_sentence.append(word_to_index[word])
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
    new_relations_list.append(substitute_sentence)

In [9]:
print(len(new_facts_list))
print(len(new_questions_list))
print(len(new_info_list))
print(len(new_relations_list))

80
80
80
80


In [10]:
one_hot_facts = []
one_hot_questions = []
one_hot_info = []
one_hot_relations = []

one_hot_length = len(word_to_index)

for sentence in new_facts_list:
    substitute_sentence = []
    for word_ind in sentence:
        substitute_word = []
        for i in range(one_hot_length):
            if i == word_ind:
                substitute_word.append(1)
            else:
                substitute_word.append(0)
        substitute_sentence.append(substitute_word)
    one_hot_facts.append(substitute_sentence)

for sentence in new_questions_list:
    substitute_sentence = []
    for word_ind in sentence:
        substitute_word = []
        for i in range(one_hot_length):
            if i == word_ind:
                substitute_word.append(1)
            else:
                substitute_word.append(0)
        substitute_sentence.append(substitute_word)
    one_hot_questions.append(substitute_sentence)

for sentence in new_info_list:
    substitute_sentence = []
    for word_ind in sentence:
        substitute_word = []
        for i in range(one_hot_length):
            if i == word_ind:
                substitute_word.append(1)
            else:
                substitute_word.append(0)
        substitute_sentence.append(substitute_word)
    one_hot_info.append(substitute_sentence)
    
for sentence in new_relations_list:
    substitute_sentence = []
    for word_ind in sentence:
        substitute_word = []
        for i in range(one_hot_length):
            if i == word_ind:
                substitute_word.append(1)
            else:
                substitute_word.append(0)
        substitute_sentence.append(substitute_word)
    one_hot_relations.append(substitute_sentence)
    
vectorized_fact_list = np.asarray(one_hot_facts)
vectorized_questions = np.asarray(one_hot_questions) 
vectorized_info = np.asarray(one_hot_info)
vectorized_relations = np.asarray(one_hot_relations)

In [11]:
print(vectorized_fact_list.shape)
print(vectorized_questions.shape)
print(vectorized_info.shape)
print(vectorized_relations.shape)

(80, 100, 141)
(80, 100, 141)
(80, 100, 141)
(80, 100, 141)


In [12]:
label_words_to_index = {}
label_index_to_words = {}
label_index = 0
label_words_to_index["\t"]=label_index
label_index_to_words[label_index]="\t"
label_index += 1
label_words_to_index["<PAD>"]=label_index
label_index_to_words[label_index]="<PAD>"
new_label_list = []
decoder_input_list = [] #in this version, we will use teacher enforcing, so we need a shifted target data
label_pad_up_to = 10
for sentence in labels_list:
    substitute_sentence = []
    decoder_subs_sentence = [] 
    decoder_subs_sentence.append(label_words_to_index["\t"]) #decoder input data starts with a \t character
    sent = sentence.split()
    for char in sent: #char could be a word or group of characters
        if char in label_words_to_index.keys():
            substitute_sentence.append(label_words_to_index[char])
            decoder_subs_sentence.append(label_words_to_index[char])
        else:
            label_index += 1
            label_words_to_index[char]=label_index
            label_index_to_words[label_index]=char
            substitute_sentence.append(label_words_to_index[char])
            decoder_subs_sentence.append(label_words_to_index[char])
    for i in range(len(substitute_sentence),label_pad_up_to): #pad to regular
        substitute_sentence.append(label_words_to_index["<PAD>"])
    for i in range(len(decoder_subs_sentence),label_pad_up_to): #pad to regular
        decoder_subs_sentence.append(label_words_to_index["<PAD>"])
        
    new_label_list.append(substitute_sentence)
    decoder_input_list.append(decoder_subs_sentence)
    
#vectorize labels
vectorized_label_list = []
for sentence in new_label_list:
    sentence_vector = []
    for word in sentence:
        vector = [0]*len(label_words_to_index)
        vector[word] = 1
        sentence_vector.append(vector)
    vectorized_label_list.append(sentence_vector)

label_array = np.asarray(vectorized_label_list)

#vectorize decoder inputs
vectorized_decoder_list = []
for sentence in decoder_input_list:
    sentence_vector = []
    for word in sentence:
        vector = [0]*len(label_words_to_index)
        vector[word] = 1
        sentence_vector.append(vector)
    vectorized_decoder_list.append(sentence_vector)
    
decoder_input_data = np.asarray(vectorized_decoder_list)

In [13]:
print(label_array.shape)
print(decoder_input_data.shape)
print(len(label_words_to_index))
print(len(label_index_to_words))

(80, 10, 12)
(80, 10, 12)
12
12


In [14]:
def logits_to_text(logits, dictionary):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """    
    return ' '.join([label_index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [15]:
# TODO: Implement
learning_rate = 0.01
hidden_units = 128
latent_dim = 128 
# TODO: Build the layers

labels_index_len = len(label_index_to_words)
info_shape = vectorized_info.shape
questions_shape = vectorized_questions.shape
facts_shape = vectorized_fact_list.shape
relations_shape = vectorized_relations.shape
    
LSTM_info_inputs = Input(shape=(None, info_shape[2]))
LSTM_info = LSTM(hidden_units,return_state=True)
LSTM_info_output, info_h, info_c=LSTM_info(LSTM_info_inputs)
info_states = [info_h,info_c]
    
LSTM_questions_inputs = Input(shape=(None, questions_shape[2]))
LSTM_questions = LSTM(hidden_units,return_state=True)
LSTM_questions_output, questions_h, questions_c=LSTM_questions(LSTM_questions_inputs)
questions_states= [questions_h,questions_c]
    
LSTM_facts_inputs = Input(shape=(None, facts_shape[2]))
LSTM_facts = LSTM(hidden_units,return_state=True)
LSTM_facts_output, facts_h, facts_c=LSTM_facts(LSTM_facts_inputs)
facts_states= [facts_h,facts_c]
    
LSTM_rela_inputs = Input(shape=(None, relations_shape[2]))
LSTM_rela = LSTM(hidden_units,return_state=True)
LSTM_rela_output,rela_h,rela_c=LSTM_rela(LSTM_rela_inputs)
rela_states= [rela_h,rela_c]
    
added_h = keras.layers.Add()([info_h,questions_h,facts_h,rela_h])
added_c = keras.layers.Add()([info_c,questions_c,facts_c,rela_c])
encoder_states = [added_h,added_c]
    
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, labels_index_len))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(labels_index_len, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define and compile model as previously
model_T = Model([LSTM_info_inputs, LSTM_questions_inputs, LSTM_facts_inputs, \
                     LSTM_rela_inputs, decoder_inputs], decoder_outputs)

model_T.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#Example inputs: X1 = "There are four apples at the top of the tree." X2 = "What do you need to get the apples?"
# X3 = "Trees are high."  Label result = "Trees -> High"
#Example inputs: X1 = "There are four apples at the top of the tree." X2 = "What do you need to get the apples?"
# X3 = "Pick apples from tree."  Label result = "Apples -> On tree"
#Example inputs: X1 = "Apples->On tree, Tree->High" X2 = "What do you need to get the apples?"
# X3 = "You can go on tree with a ladder."  Label result = "Ladder"

In [16]:
#tests.test_model_final(model_final)
batch_size = 80

model_T.fit([vectorized_info, vectorized_questions, \
                        vectorized_fact_list, vectorized_relations, decoder_input_data], \
                       label_array, batch_size=batch_size, epochs=400, validation_split=0.1)

print("model fit")



Train on 72 samples, validate on 8 samples
Epoch 1/400
72/72 [==============================] - 11s 155ms/step - loss: 2.4496 - acc: 0.0056 - val_loss: 2.1012 - val_acc: 0.8250
Epoch 2/400
72/72 [==============================] - 3s 38ms/step - loss: 2.1761 - acc: 0.7111 - val_loss: 1.4683 - val_acc: 0.8250
Epoch 3/400
72/72 [==============================] - 3s 38ms/step - loss: 1.6543 - acc: 0.7111 - val_loss: 0.7022 - val_acc: 0.8250
Epoch 4/400
72/72 [==============================] - 3s 36ms/step - loss: 1.1788 - acc: 0.7111 - val_loss: 0.8736 - val_acc: 0.8750
Epoch 5/400
72/72 [==============================] - 3s 38ms/step - loss: 1.0392 - acc: 0.7736 - val_loss: 0.5665 - val_acc: 0.8750
Epoch 6/400
72/72 [==============================] - 3s 38ms/step - loss: 0.8632 - acc: 0.7736 - val_loss: 0.6898 - val_acc: 0.8000
Epoch 7/400
72/72 [==============================] - 3s 35ms/step - loss: 0.8350 - acc: 0.7389 - val_loss: 0.5479 - val_acc: 0.8250
Epoch 8/400
72/72 [============

In [17]:
hidden_units = 128
latent_dim = 128 
labels_index_len = len(label_index_to_words)
    
encoder_model = Model([LSTM_info_inputs,LSTM_questions_inputs,LSTM_facts_inputs,LSTM_rela_inputs],encoder_states)

decoder_inputs = Input(shape=(1, labels_index_len))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_dense = Dense(labels_index_len, activation='softmax')

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [18]:
def decode_sequence(info_input, facts_input, ques_input, rela_input,label_words_to_index=label_words_to_index, \
                                        label_index_to_words=label_index_to_words, label_pad_up_to=label_pad_up_to):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict([info_input, ques_input, facts_input, rela_input])
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len(label_words_to_index)))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, label_words_to_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = label_index_to_words[sampled_token_index]
        print("sampled char ", sampled_char)
        decoded_sentence += sampled_char + " "

        # Exit condition: hit max length
        if len(decoded_sentence) > label_pad_up_to * 4:
            print("len of sentence = ", len(decoded_sentence))
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, len(label_words_to_index)))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [34]:
test_info =  vectorized_info[25,:,:]
test_info = test_info.reshape(1,*test_info.shape) 
test_ques = vectorized_questions[25,:,:]
test_ques = test_ques.reshape(1,*test_ques.shape)
test_fact =  vectorized_fact_list[25,:,:]
test_fact = test_fact.reshape(1,*test_fact.shape) 
test_rela =  vectorized_relations[25,:,:]
test_rela = test_info.reshape(1,*test_rela.shape) 

decoded_sentence = decode_sequence(test_info, test_fact, test_ques, test_rela)
check_states_value = encoder_model.predict([test_info, test_ques, test_fact, test_rela])
print("states value ", check_states_value)

sampled char  IN4
sampled char  IN4
sampled char  IN4
sampled char  IN4
sampled char  IN4
sampled char  IN4
sampled char  IN4
sampled char  IN4
sampled char  	
sampled char  	
sampled char  	
sampled char  	
sampled char  	
len of sentence =  42
states value  [array([[ 0.52528644,  2.71062088,  2.32848549, -0.50312501,  0.56951767,
        -1.13162088,  0.34072456, -1.92812324, -0.12323909,  0.70027864,
         1.21941638, -0.80078167, -1.11370873, -2.86348057, -1.78044915,
        -2.66526699,  0.40804183, -2.39201856, -1.0227679 , -0.93778896,
         0.01403493,  0.91131139,  0.04186231, -0.96145749, -3.60099983,
         2.25232053,  0.73164302,  0.08936828, -1.19693804,  1.73914933,
         0.30691409,  1.60669506,  2.56883264, -0.70603693,  2.64822865,
        -1.90900898, -0.39705122, -0.67773235,  0.46626887, -0.69848925,
         0.25378025,  1.36123562,  3.4326098 , -0.10008374,  0.47002453,
         2.85923433, -0.35645607, -2.65078568, -0.93460202, -0.77156639,
        -

In [20]:
print(decoded_sentence)

RN2 RN2 RN2 RN2 RN2 RN2 RN2 RN2 RN2 IN5 IN5 


In [21]:
print("info ", test_info[0,1,:])
print("ques", test_ques[0,5,:])

info  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
ques [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [22]:
print(facts_list)

[['NONE'], ['NONE'], ['NONE'], ['NONE'], ['NONE'], ['NONE'], ['NONE'], ['N3', 'MEANS', 'MORE'], ['NONE'], ['NONE'], ['NONE'], ['return', 'MAKES', 'ADD', ';'], ['buy', 'MAKES', 'SUBTRACT'], ['changed', 'PAST-TENSE', 'change', ';', 'change', 'ADD-OR-SUBTRACT'], ['changed', 'PAST-TENSE', 'change', ';', 'change', 'ADD-OR-SUBTRACT'], ['NONE'], ['NONE'], ['NONE'], ['NONE'], ['NONE'], ['NONE'], ['NONE'], ['N4', 'PLURAL', 'N5'], ['N4', 'PLURAL', 'N5'], ['N1', 'PLURAL', 'N3'], ['N1', 'PLURAL', 'N3'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'A1'], ['V1', 'MAKES', 'A1'], ['V1', 'MAKES', 'A1'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V2', 'MAKES', 'ADD', ';', 'V1', 'MAKES', 'ADD'], ['V2', 'MAKES', 'ADD', ';', 'V1', 'MAKES', 'ADD'], ['V2', 'MAKES', 'ADD', ';', 'V1', 'MAKES', '

In [23]:
print(questions_list)

[['how', 'many', 'N3', 'are', 'there', '?'], ['how', 'many', 'N3', 'are', 'there', '?'], ['how', 'many', 'N5', 'are', 'in', 'N1', '?'], ['how', 'many', 'N3', 'are', 'there', '?'], ['how', 'many', 'N6', 'are', 'there', '?'], ['how', 'many', 'N2', 'are', 'there', '?'], ['how', 'many', 'N1', 'are', 'there', '?'], ['how', 'many', 'N1', 'are', 'there', '?'], ['how', 'many', 'N2', 'are', 'there', '?'], ['how', 'many', 'N2', 'are', 'in', 'first', 'N1', '?'], ['how', 'many', 'N2', 'are', 'in', 'second', 'N1', '?'], ['how', 'many', 'N1', 'are', 'there', '?'], ['how', 'many', 'N1', 'are', 'there', '?'], ['how', 'many', 'N1', 'will', 'there', 'be', 'when', 'N4', 'comes', '?'], ['how', 'many', 'N2', 'are', 'in', 'N4', '?'], ['how', 'many', 'N1', 'went', 'on', 'the', 'N3', '?'], ['how', 'many', 'N3', 'are', 'in', 'N1', '?'], ['how', 'many', 'N5', 'are', 'in', 'N1', '?'], ['how', 'many', 'N5', 'are', 'there', '?'], ['how', 'many', 'N2', 'are', 'in', 'N1', '?'], ['how', 'many', 'N1', 'are', 'there', 

In [24]:
print(relations_list)

[['N1', 'KIND-OF', 'N3', ';', 'N2', 'KIND-OF', 'N3'], ['N2', 'KIND-OF', 'N3', ';', 'N1', 'KIND-OF', 'N3'], ['N2', 'KIND-OF', 'N5', ';', 'N4', 'KIND-OF', 'N5'], ['N1', 'KIND-OF', 'N3'], ['N2', 'KIND-OF', 'N6', ';', 'N4', 'KIND-OF', 'N6'], ['NONE'], ['NONE'], ['N2', 'NOT', 'KIND-OF', 'N1'], ['NONE'], ['NONE'], ['NONE'], ['last', 'week', 'BEFORE', 'today'], ['NONE'], ['last', 'week', 'BEFORE', 'yesterday', ';', 'yesterday', 'BEFORE', 'today', ';', 'V1', 'MAKES', 'N4'], ['last', 'week', 'BEFORE', 'yesterday', ';', 'yesterday', 'BEFORE', 'today', ';', 'V1', 'MAKES', 'N4'], ['V1', 'MAKES', 'SUBSTRACT'], ['N4', 'KIND-OF', 'N1'], ['N2', 'KIND-OF', 'N5', ';', 'N3', 'KIND-OF', 'N5'], ['N2', 'KIND-OF', 'N5', ';', 'N3', 'KIND-OF', 'N5'], ['N3', 'NOT', 'KIND-OF', 'N2'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['V1', 'MAKES', 'ADD'], ['N4', 'KIND-OF', 'N6', ';', 'N5', 'KIND-OF', 'N6'], ['NONE'], ['NONE'], ['N4', 'KIND-OF

In [25]:
print(info_list)

[['there', 'are', 'IN1', 'N1', 'and', 'IN2', 'N2', '.'], ['there', 'are', 'IN1', 'N2', 'and', 'IN2', 'N1', '.'], ['in', 'the', 'N1', ',', 'there', 'are', 'IN1', 'UNITS', 'of', 'N2', ',', 'IN2', 'UNITS', 'of', 'N3', 'and', 'IN3', 'UNITS', 'of', 'N4', '.'], ['there', 'are', 'IN1', 'UNITS', 'of', 'N1', 'and', 'IN2', 'UNITS', 'of', 'N2', '.'], ['at', 'the', 'N1', ',', 'there', 'are', 'IN1', 'UNITS', 'of', 'N2', 'with', 'N3', 'and', 'IN2', 'UNITS', 'of', 'N4', 'with', 'N5', '.'], ['next', 'to', 'the', 'IN1', 'N1', ',', 'there', 'are', 'IN2', 'N2', '.', 'on', 'the', 'other', 'side', 'of', 'the', 'N3', ',', 'there', 'are', 'IN3', 'N2', '.'], ['there', 'are', 'IN1', 'N1', 'in', 'the', 'N2', 'and', 'IN2', 'N1', 'on', 'N3', '.'], ['there', 'are', 'IN1', 'N1', 'and', 'IN2', 'N2', 'from', 'the', 'N3', '.', 'the', 'N4', 'has', 'IN3', 'N1', 'already', '.'], ['in', 'one', 'N1', ',', 'there', 'are', 'IN1', 'N2', '.', 'in', 'the', 'other', 'N1', ',', 'there', 'are', 'IN2', 'N2', '.'], ['in', 'one', 'N1

In [26]:
print(labels_list)

['IN1 + IN2\n', 'IN2 + IN1\n', 'IN1 + IN3\n', 'IN1\n', 'IN1 + IN2\n', 'IN2 + IN3\n', 'IN1 + IN2\n', 'IN1 + IN3\n', 'IN1 + IN2\n', 'IN1\n', 'IN2\n', 'IN1 + IN2\n', 'IN1 - IN2\n', 'IN1 + IN2\n', 'IN1 - IN2\n', 'IN1 - IN2\n', 'IN1 + IN3\n', 'IN1 + IN2 + IN3\n', 'IN1 + IN2 + IN3\n', 'IN1\n', 'IN3 + IN4\n', 'IN3 + IN4\n', 'IN3\n', 'IN4\n', 'IN1 * IN2\n', 'IN2\n', 'IN1 + IN2\n', 'IN1\n', 'IN2\n', 'IN1 + IN2\n', 'IN1\n', 'IN2\n', 'IN1 + IN2 + IN3 + IN4\n', 'IN2 + IN4\n', 'IN1 + IN3\n', 'IN1\n', 'IN2\n', 'IN3\n', 'IN4\n', 'IN1 + IN2\n', 'IN3 + IN4\n', 'IN1 + IN2 + IN3 + IN4\n', 'IN1 + IN2 + IN3 + IN4\n', 'IN1 + IN2 + IN3 + IN4\n', 'IN1 + IN2 + IN3 + IN4\n', 'IN1 + IN3\n', 'IN2 + IN4\n', 'IN1 + IN3\n', 'IN2 + IN4\n', 'IN1 + IN2\n', 'IN1\n', 'IN2\n', 'IN1 + IN1 + IN2\n', 'IN1\n', 'IN1 + IN2\n', 'IN2 + IN3\n', 'IN1\n', 'IN3\n', 'IN2\n', 'IN1 + IN2\n', 'IN3\n', 'IN3\n', 'IN1 + IN2\n', 'IN1 + IN2 + IN3\n', 'IN1 + IN2 + IN3\n', 'IN1 + IN2 + IN3\n', 'IN1 * RN1 + IN1 * RN2 + IN2\n', 'IN1\n', 'IN2 + IN